In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# verificação da estrutura de dados
pd.read_csv('/kaggle/input/congressional-voting-records/house-votes-84.csv').head(10)

In [ ]:
# A opção na_valeus define as respostas registradas como '?' como valores nulos
df = pd.read_csv('/kaggle/input/congressional-voting-records/house-votes-84.csv',na_values=['?'])
# usando a função raplace do pandas para alterar as resposas para binarios
df.replace(('y', 'n'), (1, 0), inplace=True)
df.head()

In [ ]:
# avaliação da estutura dos dados e os dados faltantes 
df.info()
df.isna().sum()

In [ ]:
# analise sobre as respostas 
# é interessandte avaliar que as opiniões são divergentes o que separa bem os as classes 
indice = df.drop('Class Name', axis=1).columns 
z = 0
fig, axes = plt.subplots(4, 4, figsize=(30, 15))# define uma tabela de 10 X 10 são 10 elementos para cada tipo de elemento 
for i in range(4): # renge de 0 até 4
    #indice = np.where(y_train == i)[0] # seleciona as imagens de cada tipo para ser plotada no grafico.
    for j in range(4): # plotar 4 elementos de cada ponto        
        sns.countplot(x = indice[z+j], hue='Class Name', data=df, palette='RdBu', ax=axes[i][j])
        # retirar as escalas e cordenadas da imagem 
        axes[i][j].set_yticks([])
    z = z + j + 1
plt.show()

In [ ]:
# usando a função raplace do pandas para alterar o label para 0 e 1
df.replace(('democrat', 'republican'), (0, 1), inplace=True)

In [ ]:
# dividindo as feature de analise e resposta
y = df['Class Name'].values
X = df.drop('Class Name', axis=1).values

In [ ]:
# para trabalhar como os dados faltantes
# função aplica os valores mais frequentes de cada coluna nos valores faltantes
imp = SimpleImputer(strategy="most_frequent")
X = imp.fit_transform(X)

In [ ]:
# avaliação base de alguns modelos de classificação
def evaluate_model(X, y, model):
    K = 5
    R = 3
    # validação cruzada
    cv = RepeatedStratifiedKFold(n_splits=K, n_repeats=R, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

# seleção de modelos 
def get_models():
    models, names = list(), list()
    # SVM
    models.append(LogisticRegression())
    names.append('LG')
    
    models.append(KNeighborsClassifier())
    names.append('KNN')
    
    models.append(RandomForestClassifier(n_estimators=1000))
    names.append('RF')
    
    models.append(ExtraTreesClassifier(n_estimators=1000))
    names.append('ET')
    
    return models, names

# avaiação do modelo AUC
def evaluate(model, test_features, test_labels):
    probs_votos = model.predict_proba(test_features)
    accuracy = roc_auc_score(test_labels, probs_votos[:,1])
    return accuracy

In [ ]:
#Avaliação dos modelos 
models, names = get_models()

results = list()

for i in range(len(models)):
    scores = evaluate_model(X, y, models[i])
    results.append(scores)
    print('>%s %.3f (%.3f)' % (names[i], np.mean(scores), np.std(scores)))

plt.boxplot(results, labels=names, showmeans=True)
plt.show()

In [ ]:
# após avaliação inicial vamos realizar um ajuste dos hiperparâmetros dos dois melhores modelos atráves do GridSearch
# divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

In [ ]:
#grid search do random forest
# definição de cada paramentro
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

# Criação do grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
rf = RandomForestClassifier(random_state = 42)
# avaliação cruzada utilizando 3 folds, 
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit
rf_random.fit(X_train, y_train)
print('Melhores parametros:',rf_random.best_params_)
# seleção do melhor modelo e avaliação da acuracia do modelo 
best_random = rf_random.best_estimator_
#fazendo a predição nos arquivos de teste
y_pred = best_random.predict(X_test)
# avaliação da acuracia do modelo
acc_vote = accuracy_score(y_test, y_pred)
print("acuracia: ",acc_vote)

In [ ]:
#metirca de avaliação da curva roc
print("AUC:" ,evaluate(best_random, X_test, y_test))

In [ ]:
# definindo hiperparâmetros do grid
grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
print(grid)

In [ ]:
# Regreção logistica para classificação 
# definindo hiperparâmetros do grid
grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

In [ ]:
print("melhores parametros :",logreg_cv.best_params_)
print("acurácia com dados de treino:",logreg_cv.best_score_)

In [ ]:
# selecionando o melhor modelo, fazendo a predição e 
best_lot = logreg_cv.best_estimator_
y_pred_log = best_lot.predict(X_test)
acc_log_vote = accuracy_score(y_test, y_pred_log)
print("accuracy_score:",acc_log_vote)

In [ ]:
#metirca de avaliação da curva roc
print("AUC:" ,evaluate(best_lot, X_test, y_test))

Os dois modelos performaram muito bem, pois a divergencia de ideias entre os grupos divide muito bem os dados faciliando a criação de um modelo com boa acurácia.
